In [171]:
from __future__ import print_function
import datetime
import pickle
import os.path
from googleapiclient.discovery import build
from google_auth_oauthlib.flow import InstalledAppFlow
from google.auth.transport.requests import Request

# If modifying these scopes, delete the file token.pickle.
SCOPES = ['https://www.googleapis.com/auth/calendar.readonly']


def main():
    """Shows basic usage of the Google Calendar API.
    Prints the start and name of the next 10 events on the user's calendar.
    """
    creds = None
    # The file token.pickle stores the user's access and refresh tokens, and is
    # created automatically when the authorization flow completes for the first
    # time.
    if os.path.exists('token.pickle'):
        with open('token.pickle', 'rb') as token:
            creds = pickle.load(token)
    # If there are no (valid) credentials available, let the user log in.
    if not creds or not creds.valid:
        if creds and creds.expired and creds.refresh_token:
            creds.refresh(Request())
        else:
            flow = InstalledAppFlow.from_client_secrets_file(
                'credentials.json', SCOPES)
            creds = flow.run_local_server(port=0)
        # Save the credentials for the next run
        with open('token.pickle', 'wb') as token:
            pickle.dump(creds, token)

    service = build('calendar', 'v3', credentials=creds)

    # Call the Calendar API
    now = datetime.datetime.utcnow().isoformat() + 'Z' # 'Z' indicates UTC time
    print('Getting the upcoming 10 events')
    events_result = service.events().list(calendarId='primary', timeMin=now,
                                        maxResults=10, singleEvents=True,
                                        orderBy='startTime').execute()
    events = events_result.get('items', [])

    if not events:
        print('No upcoming events found.')
    for event in events:
        start = event['start'].get('dateTime', event['start'].get('date'))
        print(start, event['summary'])


if __name__ == '__main__':
    main()

Getting the upcoming 10 events
2020-04-05T09:00:00-05:00 "Black Monday" (2019)
2020-04-09T09:00:00-05:00 "Brooklyn Nine-Nine" (2013)
2020-04-09T09:00:00-05:00 "Tacoma FD" (2019)
2020-04-12T09:00:00-05:00 "Black Monday" (2019)
2020-04-16T09:00:00-05:00 "Brooklyn Nine-Nine" (2013)
2020-04-19T09:00:00-05:00 "Duncanville" (2020)
2020-04-19T09:00:00-05:00 "Bob's Burgers" (2011)
2020-04-22T11:00:00-05:00 Pay credit card bill
2020-04-23T09:00:00-05:00 "Brooklyn Nine-Nine" (2013)
2020-04-23T11:00:00-05:00 Check chase bill


In [172]:
SCOPES = ['https://www.googleapis.com/auth/calendar']

creds = None
    # The file token.pickle stores the user's access and refresh tokens, and is
    # created automatically when the authorization flow completes for the first
    # time.
if os.path.exists('token.pickle'):
    with open('token.pickle', 'rb') as token:
        creds = pickle.load(token)
    # If there are no (valid) credentials available, let the user log in.
if not creds or not creds.valid:
    if creds and creds.expired and creds.refresh_token:
        creds.refresh(Request())
    else:
        flow = InstalledAppFlow.from_client_secrets_file(
                'credentials.json', SCOPES)
        creds = flow.run_local_server(port=0)
        # Save the credentials for the next run
    with open('token.pickle', 'wb') as token:
        pickle.dump(creds, token)

service = build('calendar', 'v3', credentials=creds)

In [169]:
from imdb import IMDb
from imdb.helpers import sortedEpisodes
from itertools import dropwhile, islice, takewhile
from datetime import date
from time import sleep, strptime
import locale

locale.setlocale(locale.LC_ALL,'')

def getEps(title):
    #strip title input by user
    title=title.strip()
    # Create IMDb object
    i=IMDb()
    #search for title
    search_results=i.search_movie(title)
    # Get first search result that is a TV series
    search_results= filter(lambda s:s['kind']=='tv series',search_results)
    search_results=list(islice(search_results,0,1))
    #if no result found
    if len(search_results)==0: 
        return 'No TV series matches were found for "%s".'%title
    s=search_results[0]
    i.update(s,'episodes')
    s_title=s['long imdb title']
    #if no episode info found
    if (not s.has_key('episodes')) or len(s['episodes'])==0: 
        return 'Episode info is unavailable for %s.'%s_title
    s=sortedEpisodes(s)
    if len(s)==0: 
        return 'Episode info is unavailable for %s.'%s_title
    s.reverse()
    date_today=date.today()
    e=[]
    for ep_ind in range(0, len(s)):
        if s[ep_ind].has_key('original air date'):
            if (len(s[ep_ind]['original air date'])) == 4:
                s[ep_ind]['date']=strptime(s[ep_ind]['original air date'],'%Y')
            else:
                s[ep_ind]['date']=strptime(s[ep_ind]['original air date'].replace('.',''),'%d %b %Y')
        if s[ep_ind].has_key('date'):
            s[ep_ind]['date']=date(*s[ep_ind]['date'][0:3])
            s[ep_ind]['age']=(date_today - s[ep_ind]['date']).days
            if s[ep_ind]['age']>0:
                s[ep_ind]['has aired']=True
            else:
                s[ep_ind]['has aired']=False
                e.append(s[ep_ind])
        else:
            s[ep_ind]['has aired']=False
            e.append(s[ep_ind])
    #function to get season episode format for description 
    def getSE(e):
        if not isinstance(e['season'],int): 
            return ''
        Sstr='S'+str(e['season']).zfill(2)
        Estr='E'+str(e['episode']).zfill(2)
        return ' ('+Sstr+Estr+')'
    #function to get age of episode (negative if has not aired, positive if has aired)
    def getAge(e): 
        return locale.format('%i',abs(e['age']),grouping=True)
    #function to get date of episode 
    def getDate(e): 
        return e['date'].strftime('%a, ')+str(e['date'].day)+e['date'].strftime(' %b %y')
    titles = []
    descriptions = []
    dates = []
    for i in e:
        e_out=''
        if len(e)>0:
            e=i
            titles.append(s_title)
            descriptions.append(getSE(e))
            e_out=e_out+'The next upcoming episode '+ 'for '+s_title+ ' ' +'is "'+e['title']+'"'+getSE(e)+'.'
            if e.has_key('age'):
                e_schedule= 'in %s days'%getAge(e)
                e_out=e_out+' It airs '+e_schedule+', '+getDate(e)+'.'
                dates.append(getDate(e))
            else:
                e_out=e_out+' Its air date is unavailable.'
                dates.append('unknown')
        print(e_out)
    return titles, descriptions, dates



In [170]:
def add_events(shows):
    for show in shows:
        titles, descriptions, dates=getEps(show)
        if len(titles)>0:
            for i in range(0, len(titles)):
                if dates[i] != 'unknown':
                    title = titles[i]
                    description = descriptions[i]
                    air_date = dates[i]
                    date_time = datetime.datetime.strptime(air_date, '%a, %d %b %y').strftime('%Y-%m-%d')
                    start = date_time + 'T07:00:00-07:00'
                    end = date_time + 'T22:00:00-07:00'
                    event = {
                      'summary': title,
                      'description': description,
                      'start': {
                        'dateTime': start,
                        'timeZone': 'America/New_York',
                      },
                      'end': {
                        'dateTime': end,
                        'timeZone': 'America/New_York',
                      },
                    }
                    event = service.events().insert(calendarId='primary', body=event).execute()
            print('Events created for %s' % (show))
        else:
            print('No upcoming episode information for %s' % (show))
    return 'All events created'

In [168]:
shows = ["Bob's Burgers", "Black Monday", "Saturday Night Live", "Duncanville", "Barry", "Brooklyn Nine-Nine", "Tacoma FD"]
add_events(shows)

C:\Users\Jonathan\Anaconda3\lib\site-packages\ipykernel_launcher.py:53: DeprecationWarning: This method will be removed in a future version of Python. Use 'locale.format_string()' instead.


The next upcoming episode for "Bob's Burgers" (2011) is "Episode #10.22" (S10E22). It airs in 42 days, Sun, 17 May 20.
The next upcoming episode for "Bob's Burgers" (2011) is "Episode #10.21" (S10E21). It airs in 35 days, Sun, 10 May 20.
The next upcoming episode for "Bob's Burgers" (2011) is "Episode #10.20" (S10E20). It airs in 28 days, Sun, 3 May 20.
The next upcoming episode for "Bob's Burgers" (2011) is "The Handyman Can" (S10E19). It airs in 21 days, Sun, 26 Apr 20.
The next upcoming episode for "Bob's Burgers" (2011) is "Tappy Tappy Tappy Tap Tap Tap" (S10E18). It airs in 14 days, Sun, 19 Apr 20.
Events created for Bob's Burgers
The next upcoming episode for "Black Monday" (2019) is "Arthur Ponzarelli" (S02E06). It airs in 7 days, Sun, 12 Apr 20.
The next upcoming episode for "Black Monday" (2019) is "Violent Crooks and Cooks of Books" (S02E05). It airs in 0 days, Sun, 5 Apr 20.
Events created for Black Monday
No upcoming episode information for Saturday Night Live
The next upco

'All events created'